<center>
<h1><b>Lab 2</b></h1>
<h1>PHYS 580 - Computational Physics</h1>
<h2>Professor Molnar</h2>
</br>
<h3><b>Ethan Knox</b></h3>
<h4>https://www.github.com/ethank5149</h4>
<h4>ethank5149@gmail.com</h4>
</br>
</br>
<h3><b>September 11, 2020</b></h3>
</center>

## Imports

In [52]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from functools import partial

## Euler Step

In [53]:
def euler_step(f, y, t, dt, *args):
    return y + f(y, t, *args) * dt

## DSolve Function

In [54]:
def dsolve(
    fun: callable, 
    t: np.ndarray, 
    y0: np.ndarray, 
    terminate = lambda x, y : False,
    step: callable = euler_step) -> tuple:
    
    t = np.asarray(t)  # Ensure t is a Numpy array
    y = np.zeros((np.size(t), np.size(y0)))  # Create our output data container
    y[0] = y0  # Set initial condition

    dt = t[1] - t[0]  # Assume independent variable points are equidistant
    terminated_at = -1  # Index of the terminated point

    for i in range(np.size(t)-1):
        y[i+1] = euler_step(fun, y[i], t[i], dt)
        # y[i+1] = y[i] + fun(y[i], t[i]) * dt  # Step forward

        if terminate(t[i], y[i]):  # Check termination condition
            terminated_at = i  # Set termination point
            break
    
    return t[:terminated_at], y[:terminated_at, :]

## Isothermal Drag Force

In [55]:
def df_isothermal_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    b2_m: float = 4.0e-5,
    y0scale: float = 1.0e4, 
    g: float = 9.81):
    
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * np.exp(-y[1] / y0scale)
    
    return np.asarray([y[2],
                       y[3],
                       drag_factor * y[2], 
                       -g + drag_factor * y[3]])

## Adiabatic Drag Force

In [56]:
def df_adiabatic_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    b2_m: float = 4.0e-5,
    alpha: float = 6.5e-3, 
    gamma: float = 1.4,
    T_grd: float = 293,
    g: float = 9.81):
    
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * (1 - alpha * y[1]/T_grd)**(1/ (gamma-1))
    
    return np.asarray([y[2],
                       y[3],
                       drag_factor * y[2], 
                       -g + drag_factor * y[3]])

## No Drag Force

In [57]:
def df_no_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    g: float = 9.81):
        
    return np.asarray([y[2],
                       y[3],
                       0, 
                       -g])

## Constant Drag Force

In [58]:
def df_constant_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    b2_m: float = 4.0e-5,
    g: float = 9.81):
    
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v
    
    return np.asarray([y[2],
                       y[3],
                       drag_factor * y[2], 
                       -g + drag_factor * y[3]])

In [59]:
# assumes: i) drag force F = - B_2 * v^2
#          ii) isothermal air density model: rho(y) = rho(0) * exp(-y / y0)

## Small 'shoot' Function Result Class

In [60]:
class ShootResults:
    def __init__(self, y, xmax):
        self.x = y[:,0]
        self.y = y[:,1]
        self.dx = y[:,2]
        self.dy = y[:,3]

        self.i_ymax = np.argmax(self.y)

        self.max_range = xmax
        self.max_height = self.y[self.i_ymax]
        self.x_at_max_height = self.x[self.i_ymax]

## Shoot Trajectory

In [61]:
def shoot(
    v0: float = 700., 
    theta: float = 45., 
    dt: float = 0.01,
    g: float = 9.81,
    forcing: callable = df_no_drag,
    **kwargs) -> ShootResults:

    forcing = partial(forcing, **kwargs)  # Fill in any specific parameters for the forcing methods

    vx0 = v0 * np.cos(theta / 180. * np.pi)  # Convert initial conditions into an initial velocity
    vy0 = v0 * np.sin(theta / 180. * np.pi)  # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    y0 = np.asarray([0., 0., vx0, vy0])  # Set initial condition

    maxr = v0**2 / g   # max range of shell, in vaccuum (for automatic horizontal plot range)
    maxt = maxr / y0[2]  # flight time at maxr, in vacuum (for atomatic calculation end time)

    nsteps = np.round(maxt / dt)   # time steps
    t = np.linspace(0, maxt, nsteps)

    t, y_soln = dsolve(forcing, t, y0,
        terminate=lambda _x, _y : _x > 0 and _y[1] <= 0.)

    x, y = y_soln[:,0], y_soln[:,1]
    dx, dy = y_soln[:,2], y_soln[:,3]

    xmax = (y[-1] * x[-2] - y[-2] * x[-1] ) / (y[-1] - y[-2])
    x[-1] = xmax
    y[-1] = 0.

    i_ymax = np.argmax(y)

    return ShootResults(y, xmax, i_ymax)

## Defect Function

In [62]:
def defect_func(
        angle: float,
        **kwargs) -> float:

    result = shoot(theta = angle, **kwargs)
    return result.max_range

## Sweep Optimum Angle

In [63]:
def maxes(angle):
    result_no_drag = shoot(theta=angle, forcing=df_no_drag)
    result_constant_drag = shoot(theta=angle, forcing=df_constant_drag)
    result_adiabatic_drag = shoot(theta=angle, forcing=df_adiabatic_drag)
    result_isothermal_drag = shoot(theta=angle, forcing=df_isothermal_drag)

    return result_no_drag.max_range, \
           result_constant_drag.max_range, \
           result_adiabatic_drag.max_range, \
           result_isothermal_drag.max_range

## Plot Max Ranges

In [64]:
angles = np.linspace(5, 85, 1000)

data = np.asarray([maxes(angle) for angle in angles])
no_drag, constant_drag, adiabatic_drag, isothermal_drag = data[:,0], data[:,1], data[:,2], data[:,3]

plt.xlabel('angle [degrees]')    # horizontal position
plt.ylabel('range [m]')    # vertical position
plt.suptitle("Ranges")
plt.plot(angles, no_drag, label = "No Drag")
plt.plot(angles, constant_drag, label = "Constant Drag")
plt.plot(angles, adiabatic_drag, label = "Adiabatic Drag")
plt.plot(angles, isothermal_drag, label = "Isothermal Drag")
plt.legend()             # create legends
plt.grid()
plt.show()               # show plot


IndexError: too many indices for array

In [ ]:
max_no_drag = np.argmax(no_drag)
max_constant_drag = np.argmax(constant_drag)
max_adiabatic_drag = np.argmax(adiabatic_drag)
max_isothermal_drag = np.argmax(isothermal_drag)

print(angles[max_no_drag], no_drag[max_no_drag])
print(angles[max_constant_drag], constant_drag[max_constant_drag])
print(angles[max_adiabatic_drag], adiabatic_drag[max_adiabatic_drag])
print(angles[max_isothermal_drag], isothermal_drag[max_isothermal_drag])

In [ ]:
#theta=44.95995995995996
#theta=38.793793793793796
#theta=43.67867867867868
#theta=45.92092092092092

no_drag = shoot(forcing=df_no_drag)
constant_drag = shoot(forcing=df_constant_drag)
adiabatic_drag = shoot(forcing=df_adiabatic_drag)
isothermal_drag = shoot(forcing=df_isothermal_drag)

fig = plt.figure(figsize=(16,9), constrained_layout=True)
plt.xlabel('x [m]')    # horizontal position
plt.ylabel('y [m]')    # vertical position
plt.suptitle("Trajectories")
plt.plot(no_drag.x, no_drag.y, label = "No Drag")
plt.plot(constant_drag.x, constant_drag.y, label = "Constant Drag")
plt.plot(adiabatic_drag.x, adiabatic_drag.y, label = "Adiabatic Drag")
plt.plot(isothermal_drag.x, isothermal_drag.y, label = "Isothermal Drag")
plt.legend()
plt.grid()
plt.show()

In [ ]:
def bisect_max(angles, ranges, epsilon, f):
   if not ranges:
       ranges = [f(angle) for angle in angles]

   angle1, angle2, angle3 = angles
   range1, range2, range3 = ranges
   
   assert angle1 < angle2 < angle3
   assert range1 <= range2 and range3 <= range2
   
   if angle3 - angle1 < epsilon: return angle2

   angle1_ = (angle1 + angle2) * 0.5
   angle3_ = (angle2 + angle3) * 0.5
   range1_ = f(angle1_)
   range3_ = f(angle3_)

   if range1_ < range1 and range3_ < range2:
       return bisect_max([angle1_, angle2, angle3_], [range1_, range2, range3_], epsilon, f)
   elif range1_ >= range2:
       return bisect_max([angle1, angle1_, angle2], [range1, range1_, range2], epsilon, f)
   else:
       return bisect_max([angle2, angle3_, angle3], [range2, range3_, range3], epsilon, f)